#### https://catboost.ai/en/docs/features/visualization_jupyter-notebook
#### Perform cross-validation


In [111]:
import pandas as pd
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import datetime

pd.set_option("display.max_columns",None )

class Potential_User:
    
    df_tourists = pd.DataFrame([])
    df_uselesses = pd.DataFrame([])
    
    df_tourists_columns = df_tourists.columns
    df_uselesses_columns = df_uselesses.columns
    
    train = pd.DataFrame([])
    test = pd.DataFrame([])
    train_label = []
    test_label = []
    df_Total =  pd.DataFrame([])
    df_Total_processed = []
    df_Total_processed_label = []
    
    categorical_feature_list = []
    prediction = []
    Probabilities = []
    Catboost_model = []
    
    Accuracy = -1

    def One_Hot_Enc(self, df, column):
        
        services = pd.get_dummies(df[column].str.split(',', expand=True),
                                  prefix=column, prefix_sep='').groupby(level=0, axis=1).max()
        df = pd.concat([df, services], axis='columns')
        return df
    
        
    def preprocessf(self, df):
        
        #pivote on services
        columns = ['senfs', 'serviceusage', 'topservices' ,'topservicesexc2c', 'businesstags', 'bankscp']
        for x in columns:
            df = self.One_Hot_Enc(df, x)
            
         # numerical sex   
        cleanup_nums = {"gender": {"Male": 1, "Female": 2}}
        df = df.replace(cleanup_nums)  
            
        #fillna
        for x in df.columns:
               df[x] = df[x].fillna(0)
        
         
        # len feature
        list_len = ['topservices', 'topservicesexc2c', 'businesstags', 'serviceusage', 'senfs', 'bankscp']
        for x in list_len:
            df['len_'+x] = df[x].apply(lambda x: len(str(x).split(',')))
        
        # Age feature
        today = datetime.date.today()
        this_year = int(today.strftime("%Y"))
        
        list_age = ['firsttrsdateapplication', 'firstappinstalldate', 'birthday']
        for x in list_age:
            df['age_'+x] = this_year - df[x].str.split('T',expand = True)[0].str.split('-',expand = True)[0].fillna(0).apply(int)
            
        #drop extra features
        df = df.drop(columns = ['Unnamed: 0',
                                'uuid', 'ussdid', 
                                'email', #'name', 
                                'birthday', 'bankscp',
                                'businesstags',        
                                'firsttrsdateapplication', 'firstappinstalldate',
                                'firstprovince', #'firstregion','secondprovince', 'secondregion',
                                'maskedmobilenumber', 'maskednationalid',
                                'senfs', 'serviceusage', 'topservices', 'topservicesexc2c'])
        #integer
        cat = ['name','firstregion','secondprovince', 'secondregion']
        
        for x in [x for x in df.columns if (x not in cat)]:
               df[x] = df[x].apply(int)
        
        return df
    
    
    def split_test(self, df_X, df_Y):
        
        self.train, self.test, self.train_label, self.test_label = train_test_split(df_X, df_Y, 
                                                                                     random_state=104, 
                                                                                     test_size=0.25, shuffle=True)  
        
        return (self.train, self.test, self.train_label, self.test_label)
        

    
    def prediction_model(self):
            
        self.split_test(self.df_Total_processed, self.df_Total_processed_label)    
        
        categorical_feature_list = [ self.train.columns.get_loc('name'), self.train.columns.get_loc('firstregion'), 
                                    self.train.columns.get_loc('secondprovince'), self.train.columns.get_loc('secondregion')] 
        

        
        # Initialize CatBoostClassifier
        model = CatBoostClassifier(iterations=2,
                                   learning_rate=1,
                                   depth=2)
        
#         grid = {'learning_rate': [0.03, 0.1],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9]}
            
#         randomized_search_result = model.randomized_search(grid,
#                                                    self.train,
#                                                    self.train_label,
#                                                    plot=True)
        
        # Fit model
        self.Catboost_model = model.fit(self.train, self.train_label, categorical_feature_list)
                                        #verbose=False,plot=True)
        
        # Get predicted classes
        self.prediction = model.predict(self.test)
        
        # Get predicted probabilities for each class
        self.Probabilities = model.predict_proba(self.test)
        
        # Get predicted RawFormulaVal
        preds_raw = model.predict(self.test, prediction_type='RawFormulaVal')  
        
        self.Accuracy = 100 - sum(abs(X.prediction - X.test_label)) / X.prediction.size * 100
        
        return self.Accuracy
    
    
    def __init__(self, adress_convert, adress_notconvert):
        print("Hi Potential_User class has been initialized!")
        
        self.df_tourists = pd.read_csv(adress_convert)
        self.df_uselesses = pd.read_csv(adress_notconvert)
        self.df_Total = pd.concat([self.df_tourists, self.df_uselesses])
        self.df_Total_processed_label = np.concatenate((np.ones(self.df_tourists.shape[0]), 
                                                        np.zeros(self.df_uselesses.shape[0])), axis=None)
        self.df_Total_processed = self.preprocessf(self.df_Total)
        self.df_tourists['label'] = 1
        self.df_uselesses['label'] = 0
    
        self.prediction_model()
        
if __name__ == '__main__':
    
    adress_convert = r'/home/niloofar/Desktop/campaign analysis/df_convert_850.csv'
    adress_notconvert = r'/home/niloofar/Desktop/campaign analysis/df_notconvert_10000.csv'
    
    Potential_User_Class = Potential_User(adress_convert, adress_notconvert)
    

Hi Potential_User class has been initialized!
0:	learn: 0.2627298	total: 388ms	remaining: 388ms
1:	learn: 0.2406285	total: 413ms	remaining: 0us
